In [1]:
#spark.stop()

In [15]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window as W
from pyspark.conf import SparkConf
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import pandas as pd
from pyspark.mllib.feature import HashingTF, IDF
import os

In [16]:
conf= pyspark.SparkConf()
conf.setMaster("k8s://https://"+os.environ['KUBERNETES_SERVICE_HOST'])
 
# For hdfs configuration
conf.set("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem")
conf.set("spark.hadoop.fs.hdfs.server", "org.apache.hadoop.hdfs.server.namenode.NameNode")
conf.set("spark.hadoop.conf", "org.apache.hadoop.hdfs.HdfsConfiguration")
conf.set("spark.hadoop.dfs.nameservices", "nb")
conf.set("spark.hadoop.dfs.ha.namenodes.nb", "h1,h2")
conf.set("spark.hadoop.dfs.namenode.rpc-address.nb.h1", "h3001.ali-netbase.com:9820")
conf.set("spark.hadoop.dfs.namenode.rpc-address.nb.h2", "h3101.ali-netbase.com:9820")
conf.set("spark.hadoop.dfs.client.failover.proxy.provider.nb", "org.apache.hadoop.hdfs.server.namenode.ha.ConfiguredFailoverProxyProvider")
conf.set("spark.hadoop.fs.defaultFS", "hdfs://nb:9820")
 
# For kubernetes configration
f_name = "big_data.csv"
executor_cores=8
executor_machine_memory=32
executor_heap_memory=24
executor_instances=8
driver_cores=16
driver_memory=128
 
# request on-demand machines
conf.set("spark.kubernetes.executor.label.eci", "true")
conf.set("spark.kubernetes.executor.annotation.k8s.aliyun.com/eci-use-specs", "{}-{}Gi".format(executor_cores, executor_machine_memory))
  
# machine numbers
conf.set("spark.executor.instances", executor_instances)
conf.set("spark.kubernetes.allocation.batch.size", executor_instances)
 
  
# machine cpu numbers
conf.set("spark.kubernetes.executor.request.cores", executor_cores)
conf.set("spark.kubernetes.executor.limit.cores", executor_cores)
conf.set("spark.executor.memory", "{}g".format(executor_heap_memory))
conf.set("spark.executor.cores", executor_cores)
conf.set("spark.driver.memory", "{}g".format(driver_memory))
conf.set("spark.driver.cores", driver_cores)
  
# spark version
# conf.set("spark.kubernetes.container.image", "maven-docker.netbase.com/spark-py:v2.4.6")
# conf.set("spark.kubernetes.container.image", "docker-registry.netbase.com/de/base-images/spark-py:1.2.0-spark3.1.1")
conf.set("spark.kubernetes.container.image", "docker-registry.netbase.com/de/base-images/spark-py:1.2.0-spark3.1.1-test")
conf.set("spark.kubernetes.container.image.pullPolicy", "Always")
  
conf.set("spark.kubernetes.namespace", "jupyterhub")
conf.set("spark.driver.host", os.environ['HOSTIP'])
conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
conf.set('spark.submit.deployMode', 'client')
conf.set('spark.kubernetes.pyspark.pythonVersion', "3")
# tell spark executor act on behalf to user woot
conf.set('spark.kubernetes.executorEnv.HADOOP_USER_NAME', "woot")
conf.set('spark.executorEnv.HADOOP_USER_NAME', "woot")
 
conf.set('spark.default.parallelism', '32')
conf.setAppName( os.environ['JUPYTERHUB_CLIENT_ID'])
spark = SparkSession.builder.config(conf=conf).getOrCreate()
   
spark.sparkContext.getConf().getAll()

[('spark.app.startTime', '1645164611650'),
 ('spark.kubernetes.authenticate.driver.serviceAccountName', 'spark'),
 ('spark.master', 'k8s://https://10.122.0.1'),
 ('spark.kubernetes.container.image',
  'docker-registry.netbase.com/de/base-images/spark-py:1.2.0-spark3.1.1-test'),
 ('spark.hadoop.dfs.namenode.rpc-address.nb.h1', 'h3001.ali-netbase.com:9820'),
 ('spark.kubernetes.executor.podNamePrefix',
  'jupyterhub-user-ali40netbase-com-87ace67f0b73c8c1'),
 ('spark.kubernetes.executor.label.eci', 'true'),
 ('spark.kubernetes.pyspark.pythonVersion', '3'),
 ('spark.kubernetes.executor.limit.cores', '8'),
 ('spark.hadoop.fs.hdfs.impl', 'org.apache.hadoop.hdfs.DistributedFileSystem'),
 ('spark.sql.warehouse.dir', 'file:/home/jovyan/megaopus/spark-warehouse'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.default.parallelism', '32'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'spark-application-1645164611782'),
 ('spark.hadoop.dfs.namenode.rpc-address.nb.h2', 'h3101

In [17]:
path = 'hdfs://nb/ai-pipeline/megaopus_data/{}'.format(f_name)
_df = spark.read.options(header=True, encoding="UTF-8").csv(path)
_df = _df[_df['Sound Bite Text'].isNotNull()]
_df = _df.select(F.split(_df['Sound Bite Text'], " ").alias('word_array'))

In [123]:
T_TIME = 150
if repeat_time >= 1:
    n_to_array = F.udf(lambda n : list(range(n)), T.ArrayType(T.IntegerType()))
    __df = _df.withColumn('n', F.lit(repeat_time))
    __df = __df.withColumn('n', F.explode(n_to_array(__df.n)))
else:
    __df = _df.limit(int(_df.count() * repeat_time))
__df.show()
__df.count()

+--------------------+---+
|          word_array|  n|
+--------------------+---+
|[#ESG, News:, Som...|  0|
|[#ESG, News:, Som...|  1|
|[#ESG, News:, Som...|  2|
|[#ESG, News:, Som...|  3|
|[#ESG, News:, Som...|  4|
|[#ESG, News:, Som...|  5|
|[#ESG, News:, Som...|  6|
|[#ESG, News:, Som...|  7|
|[#ESG, News:, Som...|  8|
|[#ESG, News:, Som...|  9|
|[#ESG, News:, Som...| 10|
|[#ESG, News:, Som...| 11|
|[#ESG, News:, Som...| 12|
|[#ESG, News:, Som...| 13|
|[#ESG, News:, Som...| 14|
|[#ESG, News:, Som...| 15|
|[#ESG, News:, Som...| 16|
|[#ESG, News:, Som...| 17|
|[#ESG, News:, Som...| 18|
|[#ESG, News:, Som...| 19|
+--------------------+---+
only showing top 20 rows



132118200

In [124]:
__df.rdd.getNumPartitions()
__df.repartition(400)

DataFrame[word_array: array<string>, n: int]

In [125]:
__df.printSchema()

root
 |-- word_array: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- n: integer (nullable = true)



In [126]:
hashingTF = pyspark.ml.feature.HashingTF(inputCol="word_array", outputCol="features")
hashingTF.setNumFeatures(100000)
tf = hashingTF.transform(__df)
tf.head().features

SparseVector(100000, {585: 1.0, 4705: 1.0, 5145: 1.0, 7871: 1.0, 10266: 1.0, 13956: 1.0, 16017: 3.0, 21318: 1.0, 25209: 1.0, 26365: 1.0, 26467: 1.0, 30488: 1.0, 30578: 1.0, 31833: 1.0, 35578: 1.0, 44658: 1.0, 48540: 1.0, 51152: 1.0, 51299: 1.0, 52276: 1.0, 52671: 1.0, 55867: 1.0, 56721: 1.0, 59189: 1.0, 60364: 1.0, 62891: 1.0, 65581: 2.0, 69218: 1.0, 71266: 1.0, 75882: 1.0, 81991: 1.0, 85726: 1.0, 85870: 1.0, 90073: 1.0, 95553: 1.0, 99737: 1.0})

In [127]:
tf.cache()
idf_model = pyspark.ml.feature.IDF().setInputCol("features").setOutputCol("idf").fit(tf)
tf_idf = idf_model.transform(tf)
tf_idf.printSchema()
tf_idf.cache()

root
 |-- word_array: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- n: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- idf: vector (nullable = true)



DataFrame[word_array: array<string>, n: int, features: vector, idf: vector]

In [ ]:
from datetime import datetime
ta = datetime.now()
kmeans = KMeans(featuresCol='idf', k=5, initSteps=10).setSeed(1)
model = kmeans.fit(tf_idf)
predictions = model.transform(tf_idf)
td = datetime.now()
print('elapsed time:', (td - ta).total_seconds())

22/02/18 06:58:48 WARN DAGScheduler: Broadcasting large task binary with size 1638.4 KiB
22/02/18 06:59:38 WARN DAGScheduler: Broadcasting large task binary with size 1639.0 KiB
22/02/18 06:59:48 WARN DAGScheduler: Broadcasting large task binary with size 1639.7 KiB
22/02/18 07:00:09 WARN DAGScheduler: Broadcasting large task binary with size 1640.0 KiB
22/02/18 07:00:22 WARN DAGScheduler: Broadcasting large task binary with size 1640.0 KiB
22/02/18 07:00:38 WARN DAGScheduler: Broadcasting large task binary with size 1640.4 KiB
22/02/18 07:00:49 WARN DAGScheduler: Broadcasting large task binary with size 1640.4 KiB
22/02/18 07:01:23 WARN DAGScheduler: Broadcasting large task binary with size 1640.7 KiB
22/02/18 07:01:34 WARN DAGScheduler: Broadcasting large task binary with size 1640.7 KiB
22/02/18 07:01:48 WARN DAGScheduler: Broadcasting large task binary with size 1641.1 KiB
22/02/18 07:01:59 WARN DAGScheduler: Broadcasting large task binary with size 1641.1 KiB
22/02/18 07:02:14 WAR

In [ ]:
predictions.show()

In [ ]:
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='features', \
        metricName='silhouette', distanceMeasure='squaredEuclidean')
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

In [ ]:
predictions.groupBy('prediction').\
        count().\
        sort(F.desc('count')).\
        show(truncate=False)

In [95]:
#spark.stop()